# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Scaping list of postal codes in Toronto
website_to_scape = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(website_to_scape)
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
toronto_raw_data = df[0]

In [3]:
toronto_raw_data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
# Drop rows with a borough that is Not Assigned
toronto_data = toronto_raw_data[toronto_raw_data['Borough'] != "Not assigned"]

# Group neighborhoods with the same postal code
toronto_data.groupby(['Postal Code'])['Neighborhood'].sum()

Postal Code
M1B                                       Malvern, Rouge
M1C               Rouge Hill, Port Union, Highland Creek
M1E                    Guildwood, Morningside, West Hill
M1G                                               Woburn
M1H                                            Cedarbrae
                             ...                        
M9N                                               Weston
M9P                                            Westmount
M9R    Kingsview Village, St. Phillips, Martin Grove ...
M9V    South Steeles, Silverstone, Humbergate, Jamest...
M9W                                            Northwest
Name: Neighborhood, Length: 103, dtype: object

In [19]:
toronto_data.reset_index(inplace=True)
toronto_data = toronto_data.drop(['index'], axis=1)
toronto_data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
toronto_data.shape

(103, 3)

In [10]:
geospatial_data = pd.read_csv("Geospatial_Coordinates.csv")

In [16]:
geospatial_data.head()
toronto_geospatial_data = pd.merge(toronto_data, geospatial_data, on = 'Postal Code')

In [18]:
toronto_geospatial_data.shape

(103, 5)